In [3]:

# -*- coding: utf-8 -*-
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import xlrd
import xlwt
import pandas as pd
from pandas.core.frame import DataFrame
import numpy as np
from scipy.stats import zscore

def GM11(x, n):
    '''
    灰色预测
    x：序列，numpy对象
    n:需要往后预测的个数
    '''
    x1 = x.cumsum()  # 一次累加
    z1 = (x1[:len(x1) - 1] + x1[1:]) / 2.0  # 紧邻均值
    z1 = z1.reshape((len(z1), 1))
    B = np.append(-z1, np.ones_like(z1), axis=1)
    Y = x[1:].reshape((len(x) - 1, 1))
    # a为发展系数 b为灰色作用量
    [[a], [b]] = np.dot(np.dot(np.linalg.inv(np.dot(B.T, B)), B.T), Y)  # 计算参数
    result = (x[0] - b / a) * np.exp(-a * (n - 1)) - (x[0] - b / a) * np.exp(-a * (n - 2))
    S1_2 = x.var()  # 原序列方差
    e = list()  # 残差序列
    for index in range(1, x.shape[0] + 1):
        predict = (x[0] - b / a) * np.exp(-a * (index - 1)) - (x[0] - b / a) * np.exp(-a * (index - 2))
        e.append(x[index - 1] - predict)
    S2_2 = np.array(e).var()  # 残差方差
    C = S2_2 / S1_2  # 后验差比
    print(C)
    if C <= 0.35:
        assess = '后验差比<=0.35，模型精度等级为好'
    elif C <= 0.5:
        assess = '后验差比<=0.5，模型精度等级为合格'
    elif C <= 0.65:
        assess = '后验差比<=0.65，模型精度等级为勉强'
    else:
        assess = '后验差比>0.65，模型精度等级为不合格'
    # 预测数据
    predict = list()
    for index in range(x.shape[0] + 1, x.shape[0] + n + 1):
        predict.append((x[0] - b / a) * np.exp(-a * (index - 1)) - (x[0] - b / a) * np.exp(-a * (index - 2)))
    predict = np.array(predict)
    return {
        'a': {'value': a, 'desc': '发展系数'},
        'b': {'value': b, 'desc': '灰色作用量'},
        'predict': {'value': result, 'desc': '第%d个预测值' % n},
        'C': {'value': C, 'desc': assess},
        'predict': {'value': predict, 'desc': '往后预测%d个的序列' % (n)},
    }








In [4]:
if __name__ == '__main__':
    xlsx = xlrd.open_workbook('附件2 近5年8家转运商的相关数据.xlsx')
    # 查看所有sheet列表
    print('All sheets: %s' % xlsx.sheet_names())
    sheet1 = xlsx.sheets()[0]    # 获得第1张sheet，索引从0开始
    dic={}
    for i in range(sheet1.nrows-1):
            lis=[]
            for j in range(1,sheet1.ncols):
                lis.append(sheet1.cell(i+1,j).value)
            dic[sheet1.cell(i+1,0).value]=lis
    #初始化原始数据
    T1=dic['T1']
    T2=dic['T2']
    T3=dic['T3']
    T4=dic['T4']
    T5=dic['T5']
    T6=dic['T6']
    T7=dic['T7']
    T8=dic['T8']
    
    
    
    data = np.array(T8)
    x = data[0:240]# 输入数据
    y = range(1,25) # 需要预测的数据
    result = GM11(x, len(y))
    predict = result['predict']['value']
    predict = np.round(predict, 20)
    predict=DataFrame(predict)#这时候是以行为标准写入的
    predict=predict.T#转置之后得到想要的结果

    print('预测值:', predict)
    
    

All sheets: ['运输损耗率（%）']
0.9997936137670052
预测值:          0         1         2         3         4         5        6   \
0  0.878054  0.878235  0.878416  0.878597  0.878778  0.878959  0.87914   

         7         8         9   ...       14        15        16        17  \
0  0.879321  0.879502  0.879684  ...  0.88059  0.880772  0.880953  0.881135   

         18        19       20        21        22        23  
0  0.881316  0.881498  0.88168  0.881861  0.882043  0.882225  

[1 rows x 24 columns]


In [5]:
predict.to_excel("预测.xlsx", sheet_name='T8')

In [6]:
T1

[1.5539,
 1.639,
 0.8124,
 1.2233,
 1.1194,
 1.1572,
 1.0769,
 1.1194,
 1.9129,
 2.8528,
 2.6025,
 1.8704,
 1.7381,
 1.946,
 1.8751,
 2.031,
 2.201,
 2.3285,
 1.8657,
 2.1443,
 1.9743,
 2.0924,
 1.9932,
 1.6437,
 1.4642,
 2.0404,
 1.6248,
 1.1147,
 1.1052,
 1.2044,
 0.8455,
 1.1052,
 2.1821,
 1.9649,
 1.7854,
 1.7523,
 1.9743,
 2.2388,
 2.4277,
 2.2908,
 2.1396,
 2.3899,
 1.9318,
 1.5587,
 2.3191,
 1.5776,
 2.1066,
 2.2294,
 2.1018,
 2.645,
 1.639,
 1.28,
 1.11,
 1.1572,
 0.9163,
 1.3886,
 1.8468,
 2.2341,
 2.1538,
 2.2152,
 2.2105,
 1.9885,
 3.004,
 2.7395,
 3.2165,
 3.0087,
 2.0499,
 1.9035,
 2.3097,
 2.5175,
 2.3852,
 1.9035,
 1.9554,
 2.5033,
 1.3981,
 1.1241,
 0.7841,
 1.757,
 1.3792,
 1.45,
 3.1315,
 4.0195,
 5.0,
 5.0,
 5.0,
 1.9507,
 2.4797,
 5.0,
 2.8009,
 2.3711,
 1.7193,
 1.7004,
 1.365,
 2.0357,
 2.3522,
 2.1018,
 2.1018,
 2.3333,
 1.6531,
 1.6012,
 1.1005,
 1.1572,
 1.1525,
 1.4406,
 2.645,
 2.6072,
 2.0877,
 2.2388,
 2.5128,
 1.9979,
 2.3427,
 3.1457,
 2.3002,
 1.8421,
 1